In [2]:
import os
import torch
import numpy as np
from tqdm import tqdm
from procgen import ProcgenGym3Env
from procgen_tools import maze
from procgen_tools.models import load_policy
from procgen_tools import maze as maze_api

In [3]:

from pathlib import Path
import io

save_dir = Path("maze_linear_probe_data")
if not save_dir.is_dir():
    os.mkdir(save_dir)

In [4]:
seed = 42
wrapped_venv = maze_api.create_venv(
    num=1, start_level=int(seed), num_levels=0, num_threads = 4
)
wrapped_venv.env.get_info()[0].keys()

building procgen...done


dict_keys(['prev_level_seed', 'prev_level_complete', 'level_seed', 'rgb'])

In [5]:
# Get the procgen env directly like so
wrapped_venv.env

<CEnv lib_path=/Users/paul/Projects/procgen/procgen/.build/relwithdebinfo/libenv.dylib options={'center_agent': True, 'use_generated_assets': False, 'use_monochrome_assets': False, 'restrict_themes': False, 'use_backgrounds': True, 'paint_vel_info': False, 'distribution_mode': 1, 'env_name': 'maze', 'num_levels': 0, 'start_level': 42, 'num_actions': 15, 'use_sequential_levels': False, 'debug_mode': 0, 'rand_seed': 741687692, 'num_threads': 4, 'render_human': True, 'resource_root': '/Users/paul/Projects/procgen/procgen/data/assets/'}>